In [1]:
import pandas as pd
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import roc_curve, roc_auc_score, confusion_matrix
from sklearn.model_selection import cross_val_score, LeaveOneOut, cross_val_predict, KFold, GridSearchCV
from sklearn.inspection import permutation_importance
from pickle import dump
from collections import Counter

In [2]:
# Adjust these to address the current train data and the path where the model needs to be saved + the name
DATA_PATH = "../data/train_data/INSRTR_SAR.csv"
MODEL_PATH = "../insrtr/models/gbt_classifier_v3.pkl"

In [3]:
def encode_categories(dataframe_to_encode):
    """
    Encodes categories using cat.codes from scikit.

    Parameters
    ----------
    dataframe_to_encode: input dataframe

    Returns
    -------
    dataframe_to_encode: modified dataframe
    """

    cols_to_encode = ["Enzyme", "resi_type", "resi_dssp", "prev_resi_type", "prev_resi_dssp", "next_resi_type",
                      "next_resi_dssp", "loop_seq"]
    encoded_cols = [col + "_encoded" for col in cols_to_encode]
    # Apply astype and cat.codes to each column
    encoded = dataframe_to_encode[cols_to_encode].apply(lambda column: column.astype("category").cat.codes)
    # Use assign to create new columns in dataframe
    dataframe_to_encode = dataframe_to_encode.assign(**dict(zip(encoded_cols, encoded.T.values)))
    return dataframe_to_encode


def create_x_y(dataframe_to_split, target=""):
    """
    Splits the data into descriptive space (x) and target space (y).

    Parameters
    ----------
    dataframe_to_split: input dataframe
    target: string that represents the target column

    Returns
    -------
    x_array: descriptive space numpy array
    y_array: target space numpy array

    """
    x_array = dataframe_to_split.drop(
        ["max_fold_decrease_with_peptide", "%WT_activity", "works", "fluctuation", "relative_MSA_conservation",
         "Enzyme_encoded", "Enzyme", "resi_type", "resi_dssp", "prev_resi_type", "prev_resi_dssp",
         "next_resi_type", "next_resi_dssp", "loop_seq"], axis=1).iloc[:, :].copy().values
    y_array = dataframe_to_split[target].values.ravel()
    return x_array, y_array


def save_model(classifier, filename):
    """
    Saves a scikit-learn model to disk using the pickle module.

    Parameters
    ----------
    classifier: trained model that should be saved
    filename: where the pickle file is saved
    """
    with open(filename, "wb") as file:
        dump(classifier, file)


def get_feature_importance(original_dataframe, classifier, top_n=15):
    """
    Returns N most important features, as calculated by the permutation importance.
    The permutation feature importance is defined to be the decrease in a model score when a single feature value is randomly shuffled.
    The more the score decreases, the more important the feature is.
    Permutation importance does not reflect to the intrinsic predictive value of a feature by itself but how important this feature is for a particular model.

    Parameters
    ----------
    original_dataframe: the dataframe with original features on which the model was trained
    classifier: trained model
    top_n: number of features to be returned

    Returns
    -------
    f_imp: dataframe with top_n most important features for the model
    """
    xdf =  original_dataframe.drop(
        ["max_fold_decrease_with_peptide", "%WT_activity", "works", "fluctuation", "relative_MSA_conservation",
         "Enzyme_encoded", "Enzyme", "resi_type", "resi_dssp", "prev_resi_type", "prev_resi_dssp",
         "next_resi_type", "next_resi_dssp", "loop_seq"], axis=1).iloc[:, :].copy()
    x_arr = xdf.values
    y_arr = original_dataframe['works'].values
    result = permutation_importance(classifier, x_arr, y_arr, n_repeats=20, random_state=42)
    f_imp =  pd.DataFrame({"feature_name": xdf.columns, "importance": result.importances_mean})
    f_imp = f_imp.nlargest(top_n, "importance")
    return f_imp

In [4]:
# Read data, drop embeddings and dummy columns, filer out rows with missing target
df = pd.read_csv(DATA_PATH, usecols=lambda col: "esm2" not in col and "Unnamed" not in col).dropna(
    subset=["works"])
df = encode_categories(df)

# Separate the entire space into descriptive and target arrays
x, y = create_x_y(df, "works")

# Hyperparameter tuning scenario.
# For each fold in the leave one out cross validation, do grid search on the training dataset.
# The hyperparameter value is chosen based on maority vote.
# Initialize the lists for the grid search
best_params_gridsearch = []
scores_cv =[]
# Hyperparameters to be tuned with grid search
parameters = {"n_estimators":[400,800],
              "learning_rate":[0.1, 1, 2, 4],
              "min_samples_split":[3, 5]}
# Initialize model and fold
clf = GradientBoostingClassifier(random_state=42)
kf = KFold(n_splits=len(x), shuffle=True, random_state=42)
cv = LeaveOneOut()
clf_best = GridSearchCV(clf, parameters, cv=cv)
# Apply the grid search in each LOOCV iteration
for train_index, test_index in kf.split(x):
    x_train, x_test = x[train_index], x[test_index]
    y_train, y_test = y[train_index], y[test_index]
    clf_best.fit(x_train, y_train)
    best_params_gridsearch.append(clf_best.best_params_)
    scores_cv.append(clf_best.best_estimator_.score(x_test, y_test))

# Get the majority vote for the parameters
learning_rate_gs = Counter([d["learning_rate"] for d in best_params_gridsearch])
learning_rate = learning_rate_gs.most_common(1)[0][0]
learning_rate_percentage_approval = (learning_rate_gs.most_common(1)[0][1]/47)*100
print("Percentage of approval for learning_rate={}: {:.2f}%".format(learning_rate, learning_rate_percentage_approval))

min_samples_split_gs = Counter([d["min_samples_split"] for d in best_params_gridsearch])
min_samples_split = min_samples_split_gs.most_common(1)[0][0]
min_samples_split_percentage_approval = (min_samples_split_gs.most_common(1)[0][1]/47)*100
print("Percentage of approval for min_samples={}: {:.2f}%".format(min_samples_split, min_samples_split_percentage_approval))

# Initialize the model with the best chosen parameters
clf = GradientBoostingClassifier(n_estimators=800, random_state=42, learning_rate=learning_rate, min_samples_split=min_samples_split)
scores = cross_val_score(clf, x, y, scoring="accuracy", cv=cv, n_jobs=-1)
avg_score = scores.mean()
print("LOOCV score on entire dataset: {:.4f}".format(avg_score))

# Get predictions from LOOCV - label and probability, calculate AUC and show confusion matrix
y_pred_label = cross_val_predict(clf, x, y, cv=cv, n_jobs=-1)
conf_mat = confusion_matrix(y, y_pred_label, labels=["N", "Y"])
print("Confusion matrix: \n", conf_mat)
y_pred_proba = cross_val_predict(clf, x, y, cv=cv, method="predict_proba", n_jobs=-1)
y_pred_proba = y_pred_proba[:, 1]
auc_score = roc_auc_score(y, y_pred_proba)
print("AUC score: {:.4f}".format(auc_score))

# Retrain model on entire set and save it as pickle file
clf.fit(x, y)
save_model(clf, MODEL_PATH)

Percentage of approval for learning_rate=4: 55.32%
Percentage of approval for min_samples=3: 61.70%
LOOCV score on entire dataset: 0.6596
Confusion matrix: 
 [[17  6]
 [10 14]]
AUC score: 0.7083


In [6]:
# Get feature importance
feature_importance = get_feature_importance(df, clf, top_n=10)
feature_importance

,feature_name,importance
20,resi_active_site_num_E_max,0.291489
28,loop_sasa_A_per_res,0.172340
16,resi_active_site_num_H_avg,0.031915
26,loop_radius_gyration_A,0.031915
7,resi_burial_percent,0.029787
42,loop_seq_encoded,0.024468
33,loop_P_percent,0.009574
40,next_resi_type_encoded,0.008511
0,resi_loop_index0,0.000000
1,loop_index0,0.000000
